# BiH Political Data Collection

Voting data for the following years obtained from the [Central Election Commission BiH](https://www.izbori.ba/?Lang=6). Used to control for ethnic concentration and policy preferences.
* [2010](https://www.izbori.ba/Finalni2010/Finalni/PredsjednistvoBiH/Nivo701702.aspx)
* [2014](https://www.izbori.ba/Potvrdjeni2014/Finalni/PredsjednistvoBiH/Nivo703.aspx)
* [2018](https://www.izbori.ba/rezultati_izbora?resId=25&langId=4#/1/2/0/0/703)
* [2022](https://www.izbori.ba/Rezultati_izbora/?resId=32&langId=4#/1/1/0/0/701)

I could use AI and a headless browser to scrape for both entities in each municipality. 

Questions to answer are:
* How will I classify different parties to connect them to ethnicities?